In [19]:
import numpy as np
filename = "Gemini.txt" 
raw_text = open(filename, 'r', encoding = 'utf-8').read()
raw_text = raw_text.lower()

chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i)  for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total chars :", (n_chars))
print("Total vocab :", (n_vocab))

Total chars : 163916
Total vocab : 64


In [20]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163816


In [21]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
X = torch.tensor(dataX, dtype = torch.float32).reshape(n_patterns, seq_length,1)
X = X/float(n_vocab)
y = torch.tensor(dataY)
print(X.shape, y.shape)

torch.Size([163816, 100, 1]) torch.Size([163816])


In [22]:
import torch.utils.data as data 

class Char_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size = 1, hidden_size = 256, num_layers=1,
                            batch_first = True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(256, n_vocab)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(self.dropout(x))
        return x

In [23]:
n_epochs = 20
batch_size = 128
model = Char_model()

optimizer = optim.Adam(model.parameters(), lr = 0.1)
loss_fn = nn.CrossEntropyLoss(reduction = "sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle = True, 
                         batch_size=batch_size)

best_model = None
best_loss = np.inf

for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:

        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    loss =0
    with torch.no_grad():
        for X_batch, y_batch in loader :
            y_pred = model(X_batch)
            loss+= loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_model = model.state_dict()

        print("Epoch. %d: Cross-entropy: %.4f" %(epoch, loss))

torch.save([best_model, char_to_int], "single-char.pth")

Epoch. 0: Cross-entropy: 1027930.8125
Epoch. 1: Cross-entropy: 1011831.3750
Epoch. 2: Cross-entropy: 745619.6875
Epoch. 3: Cross-entropy: 1411356.7500
Epoch. 4: Cross-entropy: 800302.2500
Epoch. 5: Cross-entropy: 790330.9375
Epoch. 6: Cross-entropy: 724182.1875
Epoch. 7: Cross-entropy: 679144.7500
Epoch. 8: Cross-entropy: 819787.1875
Epoch. 9: Cross-entropy: 708415.3750
Epoch. 10: Cross-entropy: 817261.8750
Epoch. 11: Cross-entropy: 1216122.2500
Epoch. 12: Cross-entropy: 743504.5000
Epoch. 13: Cross-entropy: 1227626.7500
Epoch. 14: Cross-entropy: 688200.1250
Epoch. 15: Cross-entropy: 968530.2500
Epoch. 16: Cross-entropy: 1104109.5000
Epoch. 17: Cross-entropy: 1163043.0000
Epoch. 18: Cross-entropy: 918243.6875
Epoch. 19: Cross-entropy: 1228238.2500


In [29]:
seq_length = 100
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]

In [31]:
best_model, char_to_int = torch.load("single-char.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i,c) for c, i in char_to_int.items())

model.load_state_dict(best_model)
pattern = [char_to_int[c] for c in prompt]

model.eval()
print('Prompt : "%s"' %prompt)
print("hh")
with torch.no_grad():
    for i in range(1000):
        x = np.reshape(pattern, (1, len(pattern), 1))/ float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)

        prediction = model(x)
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end = "")
        pattern.append(index)
        pattern = pattern[1:]


print("Done")


Prompt : "hen,” thought she, “what would become of me? they’re dreadfully
fond of beheading people here; the g"
hh
ppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppp

In [26]:
start

58230